In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from requests.exceptions import Timeout
import time

In [2]:
headers = {
    'authority': 'curlconverter.com',
    'accept': 'textml,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30',
}

In [3]:
url = 'https://arxiv.org/list/cs/recent?show=100'
response = requests.get(url, headers=headers)

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
title_list=[]
subjects_list=[]
authors_list=[]

infos = soup.dl.find_all('dd')

for info in infos:
    title = info.find('div', {'class': 'list-title mathjax'}).text.strip().replace('Title: ','')
    subjects = info.find('div', {'class': 'list-subjects'}).text.strip().replace('Subjects: ','')
    authors = info.find('div', {'class': 'list-authors'}).text.strip().replace('\n', '').replace('Authors:','')
    title_list.append(title)
    subjects_list.append(subjects)
    authors_list.append(authors)    

In [9]:
abstracts_list=[]

papers = soup.dl.find_all('dt')
pbar = tqdm(total=len(papers))

for paper in papers:
    detail_link = paper.find('a', {'title': 'Abstract'})['href']
    detail_url = f"https://arxiv.org{detail_link}"
#     detail_response = requests.get(detail_url, headers=headers)

#     detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
#     abstract = detail_soup.find('blockquote', {'class': 'abstract mathjax'}).text.strip().replace('\n','')
#     abstracts_list.append(abstract)
#     pbar.update(1)
#     time.sleep(3)
    
    try:
        detail_response = requests.get(detail_url, headers=headers, timeout=10)
        
        if detail_response.status_code == 200:
            detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
            abstract = detail_soup.find('blockquote', {'class': 'abstract mathjax'}).text.strip()
            abstracts_list.append(abstract)
            pbar.update(1)
    except Timeout:
        abstracts_list.append("pass")
        pass


 70%|███████   | 70/100 [05:01<02:22,  4.74s/it]

In [7]:
df = pd.DataFrame({'Title': title_list, 'Subjects': subjects_list, 'Author(s)': authors_list, 'Abstract': abstracts_list})
df.to_csv('output.csv', index=False)